### Features
- View dataset in FiftyOne, and select samples for annotation

In [ ]:
import os.path as osp

import fiftyone as fo
import fiftyone.types as fot
import yaml

In [ ]:
fo.annotation_config.backends["cvat"]["segment_size"] = 300

In [ ]:
dataset_type = fot.FiftyOneDataset
config_file = "../../../config.yaml"

if osp.exists(config_file):
    print(f"Loading config from {config_file}")
    config = yaml.safe_load(open(config_file, "r"))
    dataset_dir = config["dataset_dir"]
    categories = config["categories"]
    anno_key = config["anno_key"]
    label_field = config["label_field"]
    temp_anno_field = config["temp_anno_field"]
else:
    dataset_dir = "/home/fkwong/datasets/82_truckcls/data/raw/truckcls-fiftyone"
    categories = [
        "danger_vehicle",
        "dumper",
        "dumper6",
        "others",
        "pickup",
        "truck_box",
        "unknown",
    ]
    anno_key = "cvat_annotation"
    label_field = "ground_truth"  # classification
    temp_anno_field = "temp_annotation"

In [ ]:
dataset = fo.Dataset.from_dir(
    dataset_type=dataset_type,
    dataset_dir=dataset_dir,
)
sess = fo.Session(dataset, auto=False)

In [ ]:
anno_view = dataset.select(sess.selected)

In [ ]:
if len(anno_view) > 0:
    if anno_key in dataset.list_saved_views():
        print(f"{anno_key} view existed!")
    else:
        dataset.save_view(anno_key, anno_view)
else:
    print("No sample to be annotated.")

In [ ]:
if len(anno_view) > 0:
    sess = fo.Session(anno_view, auto=False)
else:
    print("No sample to be reviewed.")

In [ ]:
if len(anno_view) > 0:
    anno_results = anno_view.annotate(
        anno_key=anno_key,
        label_field=label_field,
        label_type="classification",
        classes=categories,
        launch_editor=False,
    )
else:
    print("No sample to be annotated.")

In [ ]:
anno_view = dataset.load_annotation_view(anno_key)
anno_view.load_annotations(
    anno_key,
    dest_field=temp_anno_field,
    cleanup=False,
)

In [ ]:
sess.view = anno_view

In [ ]:
anno_results = anno_view.load_annotation_results(anno_key)
for k, v in anno_results.get_status()[label_field].items():
    if v["status"] != "completed":
        print(f"Task-{k} is not completed yet, current status: {v['status']}")
        break
else:
    anno_view.load_annotations(anno_key, dest_field=label_field, cleanup=False)

In [ ]:
anno_results = dataset.load_annotation_results(
    anno_key,
    cache=False,
)

In [ ]:
dataset.delete_sample_fields(
    [
        temp_anno_field,
    ],
    error_level=1,
)
anno_results.cleanup()
if anno_key in dataset.list_annotation_runs():
    dataset.delete_annotation_run(anno_key)
if anno_key in dataset.list_saved_views():
    dataset.delete_saved_view(anno_key)
dataset.list_saved_views()
dataset.list_annotation_runs()

In [ ]:
dataset.export(
    export_dir=dataset_dir,
    dataset_type=dataset_type,
    export_media=True,
)